In [1]:
import keras
import tensorflow as tf

2024-02-22 15:52:09.825281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
keras_model_path = "gs://wake-vision-storage/saved_models/baseline_bbox_training_run2024_02_19-04_05_02_PM/wv_mobilenetv1.keras"
keras_model = keras.saving.load_model(keras_model_path)

In [4]:
saved_model_path = "/tmp/saved_model"
!rm -rf {saved_model_path}
keras_model.export(saved_model_path, format="tf_saved_model")

INFO:tensorflow:Assets written to: /tmp/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/saved_model/assets


Saved artifact at '/tmp/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  140606163328288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163714992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163717280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163333040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163335504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163718864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163722208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163723264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163722736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163719920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140606163726784

In [7]:
# load val data
import wake_vision_loader

_, val_ds, _ = wake_vision_loader.get_wake_vision(batch_size=1)

2024-02-22 15:57:09.208995: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-02-22 15:57:18.161107: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-02-22 15:57:26.350363: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


In [8]:
def representative_dataset_gen():
    for image, _ in val_ds.take(100):
        yield [image]

In [43]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()

2024-02-22 19:48:35.731513: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-22 19:48:35.731573: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-22 19:48:35.732169: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/saved_model
2024-02-22 19:48:35.765500: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-22 19:48:35.765536: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/saved_model
2024-02-22 19:48:35.892339: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-22 19:48:37.763523: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/saved_model
2024-02-22 19:48:38.263604: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 2531434

In [21]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)

In [22]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['input_layer'], 'outputs': ['output_0']}}

In [24]:
print(interpreter.get_input_details())

[{'name': 'serving_default_input_layer:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [25]:
model = interpreter.get_signature_runner()

In [36]:
batch_size = 64
val_ds = val_ds.unbatch().batch(batch_size)

In [37]:
num_correct = 0
num_batches = 0
for image, label in val_ds:
    num_batches += 1
    probs = model(input_layer=image)["output_0"]
    num_correct += (tf.argmax(probs, axis=1).numpy() == label.numpy()).sum()

2024-02-22 16:26:06.564789: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [39]:
print("Accuracy: ", num_correct / (num_batches * batch_size))

Accuracy:  0.8820405138339921


In [40]:
miap_ds = wake_vision_loader.get_miaps(batch_size=batch_size)

2024-02-22 19:32:49.441997: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


In [42]:
for name, ds in miap_ds.items():
    num_correct = 0
    num_batches = 0
    for image, label in ds:
        num_batches += 1
        probs = model(input_layer=image)["output_0"]
        num_correct += (tf.argmax(probs, axis=1).numpy() == label.numpy()).sum()
    print(f"{name} accuracy: ", num_correct / (num_batches * batch_size))
    

2024-02-22 19:34:27.208972: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


female accuracy:  0.8763786764705882


2024-02-22 19:35:38.597235: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


male accuracy:  0.8712284482758621


2024-02-22 19:37:06.251093: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

older accuracy:  0.765625


2024-02-22 19:41:22.733379: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


age_unknown accuracy:  0.5797413793103449


2024-02-22 19:47:44.265145: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


no_person accuracy:  0.9692831364829396
